## Introduction

In this lab we will use machine learning to identify the "best" available locations to open a new ice cream shop in Manhatten.  Capstone project to coursera data science.  
author: Fred Ariemma, 6/10/2019

## Parts of the lab
Load required libraries
part 1 , Find Current Ice Cream Shops in New York
Part 2 , Find the Venues that surround Current Ice Cream Shops
Part 3 , Run Regression to use Machine Learning to find the correct venue attributes
Part 4 , Find Available retail rental property in Manhatten
Part 5 , Find Venues that surround available retail rental properties
Part 6 , Use regression to score each available location

### Import necessary Libraries

In [83]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!pip install geopy 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [84]:
CLIENT_ID = 'TCDEUJ210GL3MLABYIM5X54VU5L13T2EJ4ZBFCQ1FVBWPX4J' # your Foursquare ID
CLIENT_SECRET = 'ZCXWSE4HIR3EDYJUDZMRWHGCMHCW2AWNJEIXFHHNTVREBJQV' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 130
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TCDEUJ210GL3MLABYIM5X54VU5L13T2EJ4ZBFCQ1FVBWPX4J
CLIENT_SECRET:ZCXWSE4HIR3EDYJUDZMRWHGCMHCW2AWNJEIXFHHNTVREBJQV


<a id="item1"></a>

<hr><h2> Part 1 - Find Current Ice Cream Shops in New York</h2>
<h3> Use FourSquare Seach API to find all the Ice Cream Shops in Manhatten below 90th street

#### Set Search to Ice Cream Shop with a radius of 900 meters.  Radius needs to be small enough the the API doesn't return more then the 50 maxium. 

In [85]:
search_query = 'Ice Cream shop'
radius = 900
print(search_query)

Ice Cream shop


#### Use addresses from across the city to cover all of manahatten

In [86]:
first=1
for address in(['1590 2nd Ave, New York, NY'  ,
                '232 W 60th St, New York, NY',
                '956 2nd Ave, New York, NY',
                '570 10th Ave, New York, NY',
                '132 9th Ave, New York, NY',
                '180 Varick St, New York, NY',
                '293 E 10th St, New York, NY',
                '115 Allen St, New York',
                '111 Fulton St, New York, NY',
                '209 East 26th St, New York, NY',
                '345 Park Ave S, New York, NY',
                '71 Chambers street, New York, NY',
                '102 W 35th St, New York, NY',
                '660 Madison Ave, New York, NY',
                '170 Central Park West, New York, NY',
                '2 East 14th street, New York, NY' ]):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print(latitude, longitude)
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    df = json_normalize(venues)
    print(df.shape)
    if (first!=1):
        dataframe = pd.concat([dataframe,df], ignore_index=True, sort=False)
    else:
        dataframe = df.copy()
        first = 0
# print shape returned by each address to issure we dont go to 50.  These are all standard calls
# more need to worry about foursquare daily limits

40.7754755 -73.9532323
(5, 25)
40.7695417 -73.9836135
(12, 23)
40.7552632 -73.968022
(16, 25)
40.7590853 -73.995439
(15, 18)
40.7435663 -74.0031566
(21, 18)
40.7275546 -74.005283
(23, 24)
40.727887 -73.982007
(15, 18)
40.719613 -73.9906095
(46, 25)
40.709902 -74.0068431
(17, 25)
40.7406017 -73.9812377
(15, 17)
40.7418061 -73.9846404
(18, 17)
40.714301 -74.006392
(21, 25)
40.75049955 -73.9876134503462
(21, 18)
40.7647513 -73.9709644
(13, 18)
40.7792489 -73.974303757533
(11, 23)
40.735874 -73.993526
(21, 24)


### clean up dataframe to review categories returned

In [87]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
ice_cream_shops = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
ice_cream_shops['categories'] = ice_cream_shops.apply(get_category_type, axis=1)

# clean column names by keeping only last term
ice_cream_shops.columns = [column.split('.')[-1] for column in ice_cream_shops.columns]

ice_cream_shops.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Van Leeuwen Ice Cream,Ice Cream Shop,1623 2nd Ave,US,New York,United States,NaN,170,"[1623 2nd Ave, New York, NY 10028, United States]","[{'label': 'display', 'lat': 40.77688949682537...",40.776889,-73.952445,NaN,10028,NY,5b4f75d2829b0c002c82f08e
1,Emack & Bolio's Ice Cream,Ice Cream Shop,1564 1st Ave,US,New York,United States,btwn 81st St & 82nd St,224,"[1564 1st Ave (btwn 81st St & 82nd St), New Yo...","[{'label': 'display', 'lat': 40.77387490153934...",40.773875,-73.951617,Yorkville,10028,NY,4a6fa830f964a520bad61fe3
2,Carlton Ice Cream,Ice Cream Shop,34 E 29th St,US,Brooklyn,United States,NaN,466,"[34 E 29th St, Brooklyn, NY 11226, United States]","[{'label': 'display', 'lat': 40.77899456178011...",40.778995,-73.950227,NaN,11226,NY,4c6aab4496f176b0e457b636
3,Hot Dog & Ice Cream Cart,Food Stand,John Jay Park,US,New York,United States,NaN,738,"[John Jay Park, New York, NY 10021, United Sta...","[{'label': 'display', 'lat': 40.769433, 'lng':...",40.769433,-73.949627,NaN,10021,NY,57cdc2ab498e98eef0ce1b3e
4,New York Ice Cream,Food Truck,NaN,US,New York,United States,NaN,854,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.77890221020929...",40.778902,-73.962298,NaN,NaN,NY,53baf9d8498e286e2c596892
5,Van Leeuwen Ice Cream Truck - West,Food Truck,West Dr.,US,New York,United States,at W 67th St.,482,"[West Dr. (at W 67th St.), New York, NY 10023,...","[{'label': 'display', 'lat': 40.77316083109854...",40.773161,-73.980469,NaN,10023,NY,4d2383df71276a31f9f79e99
6,Good Humor Ice Cream,Food Truck,NaN,US,NaN,United States,NaN,377,"[New York, United States]","[{'label': 'display', 'lat': 40.76616088371371...",40.766161,-73.983276,NaN,NaN,New York,4e41608be4cd43246436731f
7,Coach Ice Cream Truck,Ice Cream Shop,58th street,US,New York,United States,8th Ave,223,"[58th street (8th Ave), New York, NY, United S...","[{'label': 'display', 'lat': 40.76770574190274...",40.767706,-73.982541,NaN,NaN,NY,4c33b8287cc0c9b66d47f29a
8,World Of Nuts & Ice Cream,Candy Store,888 8th Ave,US,New York,United States,NaN,633,"[888 8th Ave, New York, NY 10019, United States]","[{'label': 'display', 'lat': 40.76421623792790...",40.764216,-73.986260,NaN,10019,NY,513104a8e4b06aa09980164c
9,RuPaul's Drag U Ice Cream Cart,Ice Cream Shop,btwn 52nd & 53rd St.,US,New York,United States,NaN,608,"[btwn 52nd & 53rd St., New York, NY 10019, Uni...","[{'label': 'display', 'lat': 40.76507142776426...",40.765071,-73.987770,NaN,10019,NY,4fda58cb121db0da6c896179


#### we ran the addresses close together to capture all the venues but we also created duplicates, so lets drop them

In [88]:
ice_cream_shops.drop_duplicates(subset=['lat','lng'],inplace=True)
print (ice_cream_shops.shape)

(162, 16)


#### Let's see all the categories

In [316]:
#print(ice_cream_shops.groupby(['categories'])['categories'].count())

categories
American Restaurant       1
Bookstore                 1
Boutique                  1
Building                  2
Candy Store               1
Chocolate Shop            1
Clothing Store            1
Convenience Store         1
Coworking Space           1
Cupcake Shop              1
Dessert Shop              8
Falafel Restaurant        1
Food                      4
Food Stand                1
Food Truck               46
General Entertainment     1
Ice Cream Shop           82
Market                    1
Museum                    1
Office                    1
Other Great Outdoors      1
Other Nightlife           1
Snack Place               1
Name: categories, dtype: int64


#### what catagories to keep? Ice Cream shop... yes what let look at food truck and dessert shop closer

In [9]:
ice_cream_shops.loc[ice_cream_shops['categories'] == 'Food Truck'].head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
4,New York Ice Cream,Food Truck,NaN,US,New York,United States,NaN,904,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.77679506432686...",40.776795,-73.963825,NaN,NaN,NY,53baf9d8498e286e2c596892
5,Van Leeuwen Ice Cream Truck - West,Food Truck,West Dr.,US,New York,United States,at W 67th St.,482,"[West Dr. (at W 67th St.), New York, NY 10023,...","[{'label': 'display', 'lat': 40.77316083109854...",40.773161,-73.980469,NaN,10023,NY,4d2383df71276a31f9f79e99
6,Good Humor Ice Cream,Food Truck,NaN,US,NaN,United States,NaN,377,"[New York, United States]","[{'label': 'display', 'lat': 40.76616088371371...",40.766161,-73.983276,NaN,NaN,New York,4e41608be4cd43246436731f
10,Mr. Softee Ice Cream Truck,Food Truck,E 40th St.,US,New York,United States,at Park Ave,950,"[E 40th St. (at Park Ave), New York, NY 10016,...","[{'label': 'display', 'lat': 40.76203444267675...",40.762034,-73.978231,NaN,10016,NY,4c6ad2dc9669e21efd8ea951
11,Fried Ice Cream,Food Truck,NaN,US,New York,United States,NaN,889,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.76184091462880...",40.761841,-73.986415,NaN,NaN,NY,520fef3411d27d27ff651c30


In [10]:
ice_cream_shops.loc[ice_cream_shops['categories'] == 'Dessert Shop'].head(3)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
17,Jacques Torres Ice Cream Shop,Dessert Shop,Grand Central Dining Concourse,US,New York,United States,Lexington,846,"[Grand Central Dining Concourse (Lexington), N...","[{'label': 'display', 'lat': 40.75296408731566...",40.752964,-73.977589,NaN,10017,NY,5578992b498e9ec2f955bfba
38,Ice Cream Crepes,Dessert Shop,NaN,US,New York,United States,NaN,351,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.75993783382736...",40.759938,-73.991425,NaN,NaN,NY,4c7af5be93ef236a5661b40f
46,Kontera Ice Cream Truck,Dessert Shop,NaN,US,NaN,United States,NaN,887,"[New York, United States]","[{'label': 'display', 'lat': 40.752355, 'lng':...",40.752355,-73.989794,NaN,NaN,New York,4e287d45c65ba11f4c5ceef2


### Venues classified as Ice Cream Shop, Food Truck and Dessert Shop all look to be Ice Cream.  So lets limit our dataframe to only include them> Some of the lat and lng coord returned be four square are incorrect, lets insure the address city is New York

In [89]:
ice_cream_shops= ice_cream_shops.loc[ice_cream_shops['categories'].isin(['Dessert Shop','Food Truck','Ice Cream Shop'])]
print(ice_cream_shops.shape)
ice_cream_shops = ice_cream_shops.loc[ice_cream_shops['city'] == 'New York']
print(ice_cream_shops.shape)
ice_cream_shops = ice_cream_shops[ice_cream_shops['address'].notnull()]
print(ice_cream_shops.shape)
ice_cream_shops.reset_index(inplace=True)
ice_cream_shops.head()

(136, 16)
(126, 16)
(88, 16)


,index,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,0,Van Leeuwen Ice Cream,Ice Cream Shop,1623 2nd Ave,US,New York,United States,NaN,170,"[1623 2nd Ave, New York, NY 10028, United States]","[{'label': 'display', 'lat': 40.77688949682537...",40.776889,-73.952445,NaN,10028,NY,5b4f75d2829b0c002c82f08e
1,1,Emack & Bolio's Ice Cream,Ice Cream Shop,1564 1st Ave,US,New York,United States,btwn 81st St & 82nd St,224,"[1564 1st Ave (btwn 81st St & 82nd St), New Yo...","[{'label': 'display', 'lat': 40.77387490153934...",40.773875,-73.951617,Yorkville,10028,NY,4a6fa830f964a520bad61fe3
2,5,Van Leeuwen Ice Cream Truck - West,Food Truck,West Dr.,US,New York,United States,at W 67th St.,482,"[West Dr. (at W 67th St.), New York, NY 10023,...","[{'label': 'display', 'lat': 40.77316083109854...",40.773161,-73.980469,NaN,10023,NY,4d2383df71276a31f9f79e99
3,7,Coach Ice Cream Truck,Ice Cream Shop,58th street,US,New York,United States,8th Ave,223,"[58th street (8th Ave), New York, NY, United S...","[{'label': 'display', 'lat': 40.76770574190274...",40.767706,-73.982541,NaN,NaN,NY,4c33b8287cc0c9b66d47f29a
4,9,RuPaul's Drag U Ice Cream Cart,Ice Cream Shop,btwn 52nd & 53rd St.,US,New York,United States,NaN,608,"[btwn 52nd & 53rd St., New York, NY 10019, Uni...","[{'label': 'display', 'lat': 40.76507142776426...",40.765071,-73.987770,NaN,10019,NY,4fda58cb121db0da6c896179


### We now have all the ice cream shops in our dataframe

<h3>Let create a map to display the locations of the shops using Folium

In [90]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel



# add Ice Cream Shops as blue circle markers
for lat, lng, label in zip(ice_cream_shops.lat, ice_cream_shops.lng, ice_cream_shops.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<hr><h2> Part 2 - Find the Venues that surround Current Ice Cream Shops</h2>
<h3> Use FourSquare Explore API to find all venue that surround each Ice Cream Shop.<br>
    we'll assume that those neighborhood blocks contain the correct mix of venues<br> that can support an ice cream shop

<a id="item2"></a>

 ### Loop through Ice cream shop finding and venues surrounding each shop by 300 meters

#### Print each shop and number of venue to insure we are pulling the data

In [91]:
first = 1
LIMIT = 100
radius=300
for index,ice_cream in ice_cream_shops.iterrows():
    
    latitude = ice_cream['lat']
    longitude = ice_cream['lng']
    
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
    results = requests.get(url).json()
#    print('There are {} around ice_cream {}.'.format(len(results['response']['groups'][0]['items']),ice_cream['name']))
    items = results['response']['groups'][0]['items']
    
    # Now let save all the venue into a dataframe
    dataframe = json_normalize(items) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories','venue.id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # filter the category for each row, apply function get_categories
    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    
    # clean columns
    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
    dataframe_filtered['number']=index
   # print(index)
    
    if (first!=1):
        surrounding_shops = pd.concat([surrounding_shops, dataframe_filtered], ignore_index=True, sort=False)
    else:
        surrounding_shops = dataframe_filtered.copy()
        first = 0
    

<p>Lets see the shape and the first 5

In [150]:
print(surrounding_shops.shape)
print(index)
print(len(surrounding_shops['number'].unique()))

surrounding_shops.head(4)
surrounding_shops['categories'].unique()

(7637, 5)
53
88


array(['Bakery', 'Beer Store', 'Butcher', 'Italian Restaurant', 'Bar',
       'Hot Dog Joint', 'Salon / Barbershop', 'Thai Restaurant',
       'Ice Cream Shop', 'Gym', 'Gastropub', 'German Restaurant',
       'Sushi Restaurant', 'Cycle Studio', 'Coffee Shop',
       'Gym / Fitness Center', 'Mexican Restaurant',
       'Japanese Restaurant', 'Yoga Studio', 'Massage Studio',
       'Juice Bar', 'Pub', 'Wine Shop', 'Salad Place', 'Ramen Restaurant',
       'Grocery Store', 'Martial Arts Dojo', 'Pizza Place',
       'Szechuan Restaurant', 'Spa', 'Piano Bar', 'Dessert Shop',
       'Latin American Restaurant', 'Seafood Restaurant',
       'French Restaurant', 'Clothing Store', 'Diner', 'Asian Restaurant',
       'Turkish Restaurant', 'Wine Bar', "Women's Store", 'Café',
       'Health & Beauty Service', 'Bagel Shop', 'Tea Room',
       'Mediterranean Restaurant', 'Frozen Yogurt Shop',
       'Vietnamese Restaurant', 'Cosmetics Shop', 'Liquor Store',
       'Mobile Phone Shop', 'Nail Salon',

### insure ice Cream shop reutrned form foursquare "search" api also returned by foursquare "explore" api

In [93]:
for index,ice_cream in ice_cream_shops.iterrows():
    check = surrounding_shops[(surrounding_shops['number'] == index) & (surrounding_shops['id'] == ice_cream['id']) &(surrounding_shops['categories']=='Ice Cream Shop') ] 
    if (check.empty):
        # Add ice cream shop
        check = check.append({'name': ice_cream['name'], 
                              'categories': "Ice Cream Shop", 
                              'id': ice_cream['id'],
                              'number': index}, ignore_index=True)
        
        surrounding_shops = surrounding_shops.append(check,ignore_index = True )
surrounding_shops.reset_index(inplace=True)
  

#### Lets see how many categies we found

In [94]:
print('There are {} uniques venue categories.'.format(len(surrounding_shops['categories'].unique())))

There are 335 uniques venue categories.



<hr><h2> Part 3 - Run Regression to use Machine Learning to find the correct venue
    attributes to create a strong environment for an Ice Cream shop</h2><h4>ice_cream_shops has our cleaned Ice Creram Shop Locations<BR>
    surrounding_shops has all the venue that surround each ice cream shop

### use one hot encoding to expand out the categories into feature columns for ML

In [95]:
# one hot encoding
surround_onehot = pd.get_dummies(surrounding_shops[['categories']], prefix="", prefix_sep="")
#surrounding_shops.reset_index()
# add neighborhood column back to dataframe
surround_onehot['number'] = surrounding_shops['number'] 

# move neighborhood column to the first column
fixed_columns = [surround_onehot.columns[-1]] + list(surround_onehot.columns[:-1])
surround_onehot = surround_onehot[fixed_columns]
print("shape before one hot",surrounding_shops.shape)
print("shape after one hot",surround_onehot.shape)
surround_grouped = surround_onehot.groupby('number').sum().reset_index()
print("shape after grouping",surround_grouped.shape)
surround_grouped.head()


shape before one hot (7637, 5)
shape after one hot (7637, 336)
shape after grouping (88, 336)


,number,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,3,0,1,3
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,3,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,2
4,4,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,4,1,0,0,0


### We now have a count of all the feature that make up a good "spot" for an ice cream shop

### Our target for regression will be the number of ice cream shops within each feature set.  Lets see how many ice cream shops are in each location and their fequency

In [96]:
surround_grouped.groupby('Ice Cream Shop').count()

,number,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Ice Cream Shop,,,,,,,,,,,,,,,,,,,,,
1,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
2,32,32,32,32,32,32,32,32,32,32,...,32,32,32,32,32,32,32,32,32,32
3,17,17,17,17,17,17,17,17,17,17,...,17,17,17,17,17,17,17,17,17,17
4,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
5,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
6,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


### lets build X and Y for regession Model

In [97]:
Y = surround_grouped['Ice Cream Shop']
X = surround_grouped.drop(['Ice Cream Shop', 'number'], axis=1)

### import regression model and fit

In [98]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit (X, Y)
# The coefficients
#print ('Coefficients: ', regr.coef_)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

<h2>Part 4 -  Find Available retail rental property in Manhatten

<h3> use screen scrapper against www.42floors.com to find availspace

<h4>function processes the soup returned by screenscapper

In [99]:
from bs4 import BeautifulSoup
import requests

def process(source):
    global available
    soup = BeautifulSoup(source, 'lxml')

# prettify shows format hmtl text to verify scrapper worked correctly
#print(soup.prettify())

    for table in soup.find_all('span', class_='col-grow-1 min-width-0-px margin-bottom-sm col-gutter-none-sm relative'):

        table2 = table.find('span',class_='title block text-overflow text-blue')
        table3 = table.find('span',class_='text-subtle block')
        street=""
        address= table2.text.strip().split('-')
        space =""
        for i in range(len(address)-1):
            street=street+space+address[i].strip()
            space=" "
    #print(table2.prettify())
    #print(address)
        address=table3.text.strip().split(', ')

        neib=address[0]
        city=address[1]
        state=address[2]
        k = 0
#for h in table1.span.contents:
 #   print (k,h)
  #  k += 1
        address = street+", " +neib+", "+city+", "+state
        location = geolocator.geocode(address)
        if location != None and neib not in (['Harlem','Washington Heights']):
            latitude = location.latitude
            longitude = location.longitude
            available = available.append({'lat' : latitude , 'lng' : longitude, 'street':street} , ignore_index=True)
            #print(available.shape)
        
        else:
            latitude = 0
            longitude = 0
  #      print(street,"   xxx   ", neib,"  xxx   ",city,state," xx", latitude, " yy",longitude ) 

<h4>scrap three screens of the most current properties

In [100]:
available = pd.DataFrame(columns=['lat', 'lng', 'street'])
#source = requests.get('https://www.nestseekers.com/commercial-lease/manhattan/?geo=2*3*4*5*6*7*8*9*10*13*18*19*20*21*24*25*105*111*121*122*129*130*132*133*183*253&search=retail').text
source = requests.get('https://42floors.com/for-lease/retail/us/ny/new-york?max=&type=for-lease&uses%5B%5D=17&regions%5B%5D=18883&order=posted_desc&min=').text
#source = requests.get('https://www.cityfeet.com/cont/soho-new-york-ny/retail-space-for-lease?l=5-8706').text
process(source)
#print('-----------')
source = requests.get('https://42floors.com/for-lease/retail/us/ny/new-york?after=1558151148.384000%2C1574706&max=&min=&order=posted_desc&regions%5B%5D=18883&type=for-lease&uses%5B%5D=17&type=for-lease').text
process(source)
source = requests.get('https://42floors.com/for-lease/retail/us/ny/new-york?after=1556257464.103000%2C1555045&max=&min=&order=posted_desc&regions%5B%5D=18883&type=for-lease&uses%5B%5D=17&type=for-lease').text
process(source)

<h4>Lets create a map showing the available retail properties on a map

In [101]:
latitude = 40.7598219
longitude = -73.9724708
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

f=0

# add Ice Cream Shops as blue circle markers
for lat, lng, label in zip(available.lat, available.lng, available.street):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='white',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map)
    f+=1

# display map
print(f)
venues_map

205


<h4> dataframe available has the latitude and longitude of all the available properties

In [102]:
available.head()

,lat,lng,street
0,40.731063,-74.002725,253 Bleecker St
1,40.741346,-73.993702,668 6th Ave
2,40.763676,-73.970326,30 E 60th St
3,40.761105,-73.974048,14 E 55th St
4,40.717071,-74.004682,88 Leonard St


<h2> Part 5 - Find Venues that surround available retail rental properties

<h3>Loop through available rentals finding venues surrounding each shop by 300 meters

In [103]:
first = 1
LIMIT = 100
radius=300
for index,avail in available.iterrows():
    
    latitude = avail['lat']
    longitude = avail['lng']
    
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
    results = requests.get(url).json()
#    print('There are {} around ice_cream {}.'.format(len(results['response']['groups'][0]['items']),ice_cream['name']))
    items = results['response']['groups'][0]['items']
    
    # Now let save all the venue into a dataframe
    dataframe = json_normalize(items) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories','venue.id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    # filter the category for each row, apply function get_categories
    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    
    # clean columns
    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
    dataframe_filtered['number']=index
   # print(index)
    
    if (first!=1):
        surrounding_avail = pd.concat([surrounding_avail, dataframe_filtered], ignore_index=True, sort=False)
    else:
        surrounding_avail = dataframe_filtered.copy()
        first = 0


In [104]:
surrounding_avail.head()

,name,categories,id,number
0,Murray's Cheese,Cheese Shop,3fd66200f964a5203ee71ee3,0
1,Murray's Cheese Bar,Cheese Shop,500d9460c84c53f005e19eb8,0
2,sweetgreen,Salad Place,57164b8838fa4c3aecc5942c,0
3,Faicco's Italian Specialties,Sandwich Place,4a74a36af964a520fede1fe3,0
4,Carmine Street Beers,Beer Store,5217e0dd11d2b33a2d55a11c,0


#### use one hot encoding to expand out the categories into feature columns for ML like we did against the ice cream shop

In [105]:
# one hot encoding
avail_onehot = pd.get_dummies(surrounding_avail[['categories']], prefix="", prefix_sep="")
#surrounding_shops.reset_index()
# add neighborhood column back to dataframe
avail_onehot['number'] = surrounding_avail['number'] 

# move neighborhood column to the first column
fixed_columns = [avail_onehot.columns[-1]] + list(avail_onehot.columns[:-1])
avail_onehot = avail_onehot[fixed_columns]
print("shape before one hot",surrounding_avail.shape)
print("shape after one hot",avail_onehot.shape)
avail_grouped = avail_onehot.groupby('number').sum().reset_index()
print("shape after grouping",avail_grouped.shape)
avail_grouped.head()

shape before one hot (16307, 4)
shape after one hot (16307, 376)
shape after grouping (205, 376)


,number,Accessories Store,Acupuncturist,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,...,Warehouse,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,5,0,0,0,...,0,0,0,0,3,1,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,6,0
2,2,2,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,2,0,0
3,3,3,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,2,0,0
4,4,0,0,0,0,0,2,0,0,0,...,0,0,0,0,1,0,0,0,3,0


<h3> now we have a dataframe that contain all the venues that surround each available location, <br>
but the columns are the same as we created for the ice cream shops 

   <h3> here we loop through the columns from the dataframe of surrounding venues of ice cream shops<br>
    (that we use to fit the regression model). then extracted the matching columns from the <br>
    available retail dataframe prepareing the data for Machine learning

<a id="item5"></a>

In [106]:

avail_matched = pd.DataFrame()
for avail_col in(surround_grouped.columns):
    if avail_col in avail_grouped.columns:
        avail_matched[avail_col]= avail_grouped[avail_col]
    else:
        avail_matched[avail_col]= 0        
    

### Lets look at the features surrounding the available real estate

In [151]:
avail_matched.head(5)


,number,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Warehouse,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,score
0,0,0,0,0,5,0,0,0,0,0,...,0,0,0,0,3,1,0,0,0,0.681991
1,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,6,1.127235
2,2,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,-0.607270
3,3,3,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0.555714
4,4,0,0,0,2,0,0,0,0,0,...,0,0,0,0,1,0,0,0,3,1.800719


<h2>Part 6- Use regression to score each available location

In [108]:
X = avail_matched.drop(['Ice Cream Shop', 'number'], axis=1)
y_hat= regr.predict(X)
y_hat.shape


(205,)

<h3> y_hat now contains the number of ice cream shops each location can support.<br>
y_hat has the same index as avail_matched,  <br>compare y_hats value with the number of ice cream shopes already that that location

In [109]:
avail_matched['score']=0
for index,current in avail_matched['Ice Cream Shop'].iteritems():  
    score = y_hat[index]/(current+1)  # the score considering existing shops plus our new one
    avail_matched.ix[index,'score']= score
    #print("index: ",index, "  Y hat: ",y_hat[index], "  currently: ",current, "   score: ",score)

c:\python36\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


<h3>Here are our top ten locations

In [113]:
avail_matched.sort_values('score', ascending=False).head(10)
    

,number,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Warehouse,Watch Shop,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,score
114,114,0,1,0,4,0,0,0,0,0,...,0,0,0,0,1,2,0,0,0,4.176790
118,118,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.558174
62,62,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3.540091
35,35,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,3,3.377816
36,36,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,3,3.377816
40,40,0,0,0,1,0,0,0,0,0,...,0,0,0,0,2,2,0,0,1,2.904501
138,138,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,1,2,2.884596
142,142,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,1,2,2.884596
120,120,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,2.823435
53,53,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,2,0,1,1,2.750434


<a id="item6"></a>

In [141]:
latitude = 40.7598219
longitude = -73.9724708
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

for index,retail in avail_matched.sort_values('score', ascending=False).head(10).iterrows():
    #print (retail.number,index)
 
    print ("Score:",round(avail_matched.iloc[index]['score'],2),"/ Address:", available['street'].iloc[index], ", NY, NY / Index:", 
           index, "Longit.:", available.iloc[index]['lng'] )
    folium.CircleMarker(
        [available['lat'].iloc[index], available['lng'].iloc[index]],
        radius=5,
        color='white',
        popup=available['street'].iloc[index],
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map)


# display map

venues_map

Score: 4.18 / Address: 666 Greenwich St , NY, NY / Index: 114 Longit.: -74.0079219
Score: 3.56 / Address: 365 7th Ave , NY, NY / Index: 118 Longit.: -73.9920832
Score: 3.54 / Address: 714 2nd Ave , NY, NY / Index: 62 Longit.: -73.9736938
Score: 3.38 / Address: 133 W 25th St , NY, NY / Index: 35 Longit.: -73.993242021211
Score: 3.38 / Address: 133 W 25th St , NY, NY / Index: 36 Longit.: -73.993242021211
Score: 2.9 / Address: 200 West End Ave , NY, NY / Index: 40 Longit.: -73.9854488
Score: 2.88 / Address: 1311 Lexington Ave , NY, NY / Index: 138 Longit.: -73.9544378
Score: 2.88 / Address: 1311 Lexington Ave , NY, NY / Index: 142 Longit.: -73.9544378
Score: 2.82 / Address: 328 330 E 14th St , NY, NY / Index: 120 Longit.: -73.9842522857143
Score: 2.75 / Address: 45 John St , NY, NY / Index: 53 Longit.: -74.0078826


In [138]:
available.head()

,lat,lng,street
0,40.731063,-74.002725,253 Bleecker St
1,40.741346,-73.993702,668 6th Ave
2,40.763676,-73.970326,30 E 60th St
3,40.761105,-73.974048,14 E 55th St
4,40.717071,-74.004682,88 Leonard St
